In [1]:
# pip install pandas_ta

In [2]:
# import relevant libraries
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

# Download S&P500 Data

In [3]:
snp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

snp500['Symbol'] = snp500['Symbol'].str.replace('.', '-')
symbols = snp500['Symbol'].unique().tolist()

end_date = '2023-09-27'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols, start=start_date, end=end_date).stack()

df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()
df

[*********************100%***********************]  503 of 503 completed

1 Failed download:
- VLTO: Data doesn't exist for startDate = 1443499200, endDate = 1695787200


adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.588043   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361626   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.748627   27.264999   28.377501   26.965000   28.207500   
           ABBV     37.024632   52.790001   54.189999   51.880001   53.099998   
           ABT      33.807266   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 YUM     124.010002  124.010002  124.739998  123.449997  124.239998   
           ZBH     112.216316  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZION     33.990002   33.990002   34.700001   33.840000   33.840000   
           ZTS     176.869995  176.869995  178.449997  176.270004  176.580002   

                        volume  
date       ticker               
2015-09-29 A         2252400.0  
           AAL       7478800.0  
           AAPL    293461600.0  
           ABBV     12842800.0  
           ABT      12287500.0  
...                        ...  
2023-09-26 YUM       1500600.0  
           ZBH       3610500.0  
           ZBRA       355400.0  
           ZION      1586100.0  
           ZTS       1463200.0  

[994088 rows x 6 columns]

# Features and Technical Indicators

In [4]:
# calculate features and technical indicators for each stock

# Garman Klass Volatility
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

# Relative Strength Index
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

# Bollinger Band
df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

# Average True Range
def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

# Moving Average Convergence Divergence
def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

# Dollar Volume
df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.588043   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361626   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.748627   27.264999   28.377501   26.965000   28.207500   
           ABBV     37.024632   52.790001   54.189999   51.880001   53.099998   
           ABT      33.807266   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 YUM     124.010002  124.010002  124.739998  123.449997  124.239998   
           ZBH     112.216316  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZION     33.990002   33.990002   34.700001   33.840000   33.840000   
           ZTS     176.869995  176.869995  178.449997  176.270004  176.580002   

                        volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-09-29 A         2252400.0         -0.000854        NaN       NaN   
           AAL       7478800.0         -0.000443        NaN       NaN   
           AAPL    293461600.0         -0.005307        NaN       NaN   
           ABBV     12842800.0         -0.049280        NaN       NaN   
           ABT      12287500.0         -0.008237        NaN       NaN   
...                        ...               ...        ...       ...   
2023-09-26 YUM       1500600.0          0.000053  36.057176  4.826202   
           ZBH       3610500.0          0.000224  31.893246  4.751923   
           ZBRA       355400.0          0.000133  29.494977  5.400991   
           ZION      1586100.0          0.000307  46.707773  3.539073   
           ZTS       1463200.0          0.000075  42.623470  5.163569   

                     bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2015-09-29 A            NaN       NaN       NaN       NaN      71.148909  
           AAL          NaN       NaN       NaN       NaN     279.420126  
           AAPL         NaN       NaN       NaN       NaN    7262.771592  
           ABBV         NaN       NaN       NaN       NaN     475.499937  
           ABT          NaN       NaN       NaN       NaN     415.406784  
...                     ...       ...       ...       ...            ...  
2023-09-26 YUM     4.856171  4.886139  0.142547 -1.363695     186.089409  
           ZBH     4.791592  4.831260 -0.381708 -0.881067     405.157010  
           ZBRA    5.539167  5.677342 -0.057389 -1.600791      79.595386  
           ZION    3.594527  3.649982 -0.161699 -0.164625      53.911542  
           ZTS     5.222385  5.281201  0.651515 -1.188278     258.796177  

[994088 rows x 14 columns]

# Convert to monthly data, and filter top most liquid stocks

In [5]:
# creating a list of columns to keep
cols = [c for c in df.columns if c not in ['dollar_volume','volume','open','high','low','close']]

# unstacking df by ticker
unstacked_df = df.unstack('ticker')

# convert index to datetime
unstacked_df.index = pd.to_datetime(unstacked_df.index)

# resample 'dollar volume' to monthly frequency and compute mean
monthly_avg_dollar_volume = unstacked_df['dollar_volume'].resample('M').mean()

# resample the columns in 'cols' to monthly frequency and take last value of each month
monthly_last_value = unstacked_df[cols].resample('M').last()

# re-stack df back to original structure
restacked_dollar_volume = monthly_avg_dollar_volume.stack('ticker').to_frame('dollar_volume')

restacked_last_value = monthly_last_value.stack('ticker')

# concatenate the 2 dataframes above
data = pd.concat([restacked_dollar_volume,restacked_last_value],axis=1)

# drop rows with null values
data = data.dropna()

data

dollar_volume   adj close       atr   bb_high    bb_low  \
date       ticker                                                            
2015-11-30 A          136.444137   39.152691 -1.033887  3.694119  3.549210   
           AAL        287.915791   39.429928  0.190822  3.827636  3.672028   
           AAPL      4039.899032   26.960352 -0.967900  3.372114  3.285478   
           ABBV       343.971758   41.160305 -0.526809  3.841588  3.745051   
           ABT        213.736338   38.669403 -1.064842  3.709289  3.665571   
...                          ...         ...       ...       ...       ...   
2023-09-30 OTIS       156.200745   79.290001 -1.028320  4.472419  4.381831   
           ABNB      1633.500725  132.279999 -1.006939  5.024801  4.857047   
           CEG        197.815385  108.489998 -0.436215  4.738248  4.657897   
           GEHC       212.434213   66.179550 -0.893478  4.271243  4.156170   
           KVUE       692.055664   20.639999 -0.899746  3.194060  3.043960   

                     bb_mid  garman_klass_vol      macd        rsi  
date       ticker                                                   
2015-11-30 A       3.621664         -0.001810  0.567158  73.421521  
           AAL     3.749832         -0.000966 -0.418772  40.718957  
           AAPL    3.328796         -0.003027 -0.142789  55.537427  
           ABBV    3.793320         -0.053947  0.145677  49.376878  
           ABT     3.687430         -0.009962  0.335558  56.962660  
...                     ...               ...       ...        ...  
2023-09-30 OTIS    4.427125          0.000093 -1.534536  33.116257  
           ABNB    4.940924          0.000213 -0.037854  44.494127  
           CEG     4.698072          0.000274  0.366876  55.245471  
           GEHC    4.213706          0.000185 -1.116463  40.922330  
           KVUE    3.119010          0.000158 -1.435621  35.706322  

[46935 rows x 9 columns]

In [6]:
# 5 year moving average of dollar volume
rolling_mean_data = data.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean()
data['dollar_volume'] = rolling_mean_data.stack('ticker')

# ranking each ticker by daily dollar volume
data['dollar_vol_rank'] = data.groupby('date')['dollar_volume'].rank(ascending=False)

# dropping stocks with dollar_vol_rank below 150 (selecting top 150)
data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

data

adj close       atr   bb_high    bb_low    bb_mid  \
date       ticker                                                       
2016-10-31 AAL      39.134323  0.402199  3.706314  3.604673  3.655493   
           AAPL     26.316149 -1.038688  3.355806  3.298038  3.326922   
           ABBV     41.009075 -0.893132  3.880188  3.771814  3.826001   
           ABT      34.630020 -1.035224  3.665095  3.564121  3.614608   
           ACN     104.350311 -0.996806  4.668056  4.644779  4.656418   
...                       ...       ...       ...       ...       ...   
2023-09-30 WFC      40.650002 -0.558742  3.798900  3.718132  3.758516   
           WMT     162.500000 -0.196379  5.116986  5.081613  5.099300   
           XOM     116.410004  0.601335  4.793504  4.713293  4.753399   
           MRNA     98.120003 -0.529511  4.788149  4.582514  4.685332   
           ABNB    132.279999 -1.006939  5.024801  4.857047  4.940924   

                   garman_klass_vol      macd        rsi  
date       ticker                                         
2016-10-31 AAL            -0.000176  1.131595  62.203507  
           AAPL           -0.002228 -0.195978  49.891122  
           ABBV           -0.041756 -0.760594  27.477661  
           ABT            -0.006476 -0.650889  38.008858  
           ACN            -0.004026 -0.135457  53.823660  
...                             ...       ...        ...  
2023-09-30 WFC             0.000234 -0.282325  40.920274  
           WMT             0.000024  0.399459  54.722508  
           XOM             0.000045  1.400623  59.440192  
           MRNA            0.000146 -0.376899  38.747314  
           ABNB            0.000213 -0.037854  44.494127  

[12516 rows x 8 columns]

# Calculating monthly returns of different time periods

In [7]:
# function to calculate monthly returns
def calculate_returns(df):
    outlier_cutoff = 0.005
    time_period = [1,2,3,6,9,12]
    for i in time_period:
        returns = df['adj close'].pct_change(i)

        # clipping outliers based on quantiles
        lower_bound = returns.quantile(outlier_cutoff)
        upper_bound = returns.quantile(1-outlier_cutoff)
        clipped_returns = returns.clip(lower=lower_bound, upper=upper_bound)

        # convert to compound returns and annualise
        annualised_returns = ((clipped_returns + 1)**(1/i))-1

        df[f'{i}m_return'] = annualised_returns

    return df

In [8]:
data = data.groupby(level=1,group_keys=False).apply(calculate_returns).dropna()
data

adj close       atr   bb_high    bb_low    bb_mid  \
date       ticker                                                       
2017-10-31 AAL      45.534168  1.011062  3.994389  3.849110  3.921750   
           AAPL     39.870968 -0.906642  3.692324  3.598569  3.645446   
           ABBV     68.772293  0.375557  4.307973  4.215227  4.261600   
           ABT      48.969311 -1.040044  3.949284  3.902136  3.925710   
           ACN     130.375122 -0.986514  4.889487  4.810123  4.849805   
...                       ...       ...       ...       ...       ...   
2023-09-30 WFC      40.650002 -0.558742  3.798900  3.718132  3.758516   
           WMT     162.500000 -0.196379  5.116986  5.081613  5.099300   
           XOM     116.410004  0.601335  4.793504  4.713293  4.753399   
           MRNA     98.120003 -0.529511  4.788149  4.582514  4.685332   
           ABNB    132.279999 -1.006939  5.024801  4.857047  4.940924   

                   garman_klass_vol      macd        rsi  1m_return  \
date       ticker                                                     
2017-10-31 AAL            -0.000363 -0.018697  41.051793  -0.014108   
           AAPL           -0.000892 -0.039276  69.196794   0.096808   
           ABBV           -0.029822  0.473813  55.247815   0.022728   
           ABT            -0.004349  0.276133  53.844948   0.021276   
           ACN            -0.003359  0.352343  69.365269   0.064180   
...                             ...       ...        ...        ...   
2023-09-30 WFC             0.000234 -0.282325  40.920274  -0.015500   
           WMT             0.000024  0.399459  54.722508  -0.000676   
           XOM             0.000045  1.400623  59.440192   0.046947   
           MRNA            0.000146 -0.376899  38.747314  -0.132219   
           ABNB            0.000213 -0.037854  44.494127   0.005549   

                   2m_return  3m_return  6m_return  9m_return  12m_return  
date       ticker                                                          
2017-10-31 AAL      0.022981  -0.023860   0.016495   0.007008    0.012702  
           AAPL     0.015250   0.044955   0.028875   0.038941    0.035228  
           ABBV     0.098590   0.091379   0.056495   0.047273    0.044026  
           ABT      0.034308   0.034801   0.038672   0.031320    0.029294  
           ACN      0.048455   0.037203   0.028692   0.027398    0.018728  
...                      ...        ...        ...        ...         ...  
2023-09-30 WFC     -0.057917  -0.013554   0.016712   0.000702    0.003255  
           WMT      0.010014   0.012354   0.017574   0.016553    0.020256  
           XOM      0.046139   0.030496   0.012838   0.008747    0.027037  
           MRNA    -0.086803  -0.068763  -0.071952  -0.064976   -0.015431  
           ABNB    -0.067704   0.010603   0.010289   0.049124    0.019401  

[10335 rows x 14 columns]

# Fama-French Factors and Rolling Factor Betas

In [9]:
# dropped risk free rate
fama_french = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start='2010')[0].drop('RF', axis=1)
fama_french.index = fama_french.index.to_timestamp()
fama_french.index.name = 'date'
fama_french = fama_french.resample('M').last() / 100
fama_french = fama_french.join(data['1m_return']).sort_index()

In [10]:
# filtering out stocks with less than 10 months of data
observations = fama_french.groupby(level=1).size()
valid = observations[observations >= 10]
fama_french = fama_french[fama_french.index.get_level_values('ticker').isin(valid.index)]

In [11]:
# rolling factor betas
def compute_rolling_betas(df):
    y = df['1m_return']
    x = sm.add_constant(df.drop('1m_return', axis=1))
    rolling_ols = RollingOLS(endog=y, 
                             exog=x, 
                             window=min(24, df.shape[0]), 
                             min_nobs=len(df.columns)+1)
    fitted_model = rolling_ols.fit(params_only=True)
    betas = fitted_model.params.drop('const', axis=1)
    return betas

In [12]:
betas = (fama_french.groupby(level=1, group_keys=False).apply(compute_rolling_betas))

In [13]:
fama_french_factors = betas.columns.to_list()

# Join the betas to the data, shifting the betas by one period
data = data.join(betas.groupby('ticker').shift())

# Fill NaN values in the factors columns with the mean of that factor for the respective ticker
for factor in fama_french_factors:
    data[factor] = data.groupby('ticker')[factor].transform(lambda x: x.fillna(x.mean()))

# Drop the 'adj close' column
data.drop(columns='adj close', inplace=True)

# Remove any rows with NaN values
data.dropna(inplace=True)

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10022 entries, (Timestamp('2017-10-31 00:00:00'), 'AAL') to (Timestamp('2023-09-30 00:00:00'), 'MRNA')
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   atr               10022 non-null  float64
 1   bb_high           10022 non-null  float64
 2   bb_low            10022 non-null  float64
 3   bb_mid            10022 non-null  float64
 4   garman_klass_vol  10022 non-null  float64
 5   macd              10022 non-null  float64
 6   rsi               10022 non-null  float64
 7   1m_return         10022 non-null  float64
 8   2m_return         10022 non-null  float64
 9   3m_return         10022 non-null  float64
 10  6m_return         10022 non-null  float64
 11  9m_return         10022 non-null  float64
 12  12m_return        10022 non-null  float64
 13  Mkt-RF            10022 non-null  float64
 14  SMB               10022 non-null  float64
 15  HML       

In [15]:
data.to_csv('data.csv', index = True)